In [1]:
# Environment information
import sklearn
import rdkit
import pandas
import numpy

print("scikit-learn:", sklearn.__version__)
print("RDKit:", rdkit.__version__)
print("pandas:", pandas.__version__)
print("numpy:", numpy.__version__)


scikit-learn: 1.6.1
RDKit: 2025.03.2
pandas: 2.2.3
numpy: 2.2.6


In [ ]:
# This notebook reproduces the results in the manuscript:
# "Rapid and General Machine Learning Modeling of Coupling Reactions..."
# Author: Shinya Shiomi
# Python 3.10
# RDKit 2025.03.2

# ============================================================
# ElasticNet regression with 10 random splits (SI minimal)
# ============================================================

import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from scipy.stats import pearsonr

from sklearn.model_selection import KFold, ParameterGrid, train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error, r2_score


# --------------------------
# Settings
# --------------------------
INPUT_CSV = "ML_datasheet.csv"   # ← dataset file
TS = 0.30
N_SPLITS = 5
RANDOM_SEEDS = list(range(10))

ALPHAS = np.arange(0.01, 1.00, 0.05)
RATIOS = np.arange(0.01, 1.00, 0.05)
PARAM_GRID = list(ParameterGrid({"alpha": ALPHAS, "l1_ratio": RATIOS}))

KF = KFold(n_splits=N_SPLITS, shuffle=True, random_state=0)


# --------------------------
# Load data
# --------------------------
t0 = time.time()

ml = pd.read_csv(INPUT_CSV)
X = ml.iloc[:, 4:4100]
y = ml.iloc[:, 3]


# --------------------------
# Pipeline
# --------------------------
def make_pipeline(alpha=1.0, l1_ratio=0.5):
    return Pipeline([
        ("scaler", StandardScaler(with_mean=False)),
        ("model", ElasticNet(
            alpha=alpha,
            l1_ratio=l1_ratio,
            max_iter=500000,
            tol=1e-3,
            selection="random",
            random_state=0
        ))
    ])


# --------------------------
# Run one split
# --------------------------
def run_one_split(seed):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=TS, random_state=seed
    )

    pipe = make_pipeline()
    best = None

    for params in PARAM_GRID:
        pipe.set_params(model__alpha=params["alpha"], model__l1_ratio=params["l1_ratio"])
        scores = cross_val_score(
            pipe, X_train, y_train,
            cv=KF,
            scoring="neg_mean_absolute_error",
            n_jobs=-1
        )
        mean_score = float(np.mean(scores))

        if (best is None) or (mean_score > best["score"]):
            best = {
                "alpha": params["alpha"],
                "l1_ratio": params["l1_ratio"],
                "score": mean_score
            }

    pipe.set_params(model__alpha=best["alpha"], model__l1_ratio=best["l1_ratio"])
    pipe.fit(X_train, y_train)

    y_pred = pipe.predict(X_test)

    return {
        "seed": seed,
        "alpha": best["alpha"],
        "l1_ratio": best["l1_ratio"],
        "test_mae": mean_absolute_error(y_test, y_pred),
        "test_r2": r2_score(y_test, y_pred),
        "y_test": y_test.to_numpy(),
        "y_pred": y_pred
    }


# --------------------------
# Execute 10 splits
# --------------------------
results = []
for seed in tqdm(RANDOM_SEEDS):
    results.append(run_one_split(seed))

df_summary = pd.DataFrame([{
    "seed": r["seed"],
    "test_mae": r["test_mae"],
    "test_r2": r["test_r2"]
} for r in results])

mean_mae = df_summary["test_mae"].mean()


# ============================================================
# Plot 1: Test MAE bar plot
# ============================================================
plt.figure(figsize=(7, 5))
plt.bar(df_summary["seed"], df_summary["test_mae"])

plt.axhline(mean_mae, linestyle="--", label=f"Mean ({mean_mae:.2f})")

plt.xlabel("Random seed", fontsize=14)
plt.ylabel("Test MAE", fontsize=14)
plt.title("Test MAE over Ten Random Splits", fontsize=16)

plt.ylim(0, 20)                 # ← 固定
plt.yticks([0,5,10,15,20])      # ← 15まで見える

plt.grid(axis="y", linestyle="--", alpha=0.6)
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()


# ============================================================
# Plot 2: Representative scatter (seed = 0)
# ============================================================
rep = next(r for r in results if r["seed"] == 0)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TS, random_state=0
)

pipe = make_pipeline(rep["alpha"], rep["l1_ratio"])
pipe.fit(X_train, y_train)

y_pred_train = pipe.predict(X_train)
y_pred_test = pipe.predict(X_test)

plt.figure(figsize=(6.5, 6.5))

plt.scatter(y_pred_train, y_train, color="gray", alpha=0.4, label="Train")
plt.scatter(y_pred_test, y_test, color="#FF7F0E", alpha=0.9, label="Test")

plt.plot([0, 100], [0, 100], "k--")

plt.xlabel("Predicted yield (%)", fontsize=18)
plt.ylabel("Experimental yield (%)", fontsize=18)
plt.title("Predicted vs Experimental (seed = 0)", fontsize=20)

plt.xlim(0, 100)
plt.ylim(0, 100)

plt.legend(fontsize=14)
plt.grid(True, linestyle="--", alpha=0.5)

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.tight_layout()
plt.show()
